In [1]:
import sys, os
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(r"C:\Users\super\DAG")
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
print(os.getcwd())

import torch
import torch.nn as nn
from torch.optim import Adam, LBFGS
from nodag_gumbel_softmax import train_gumbel_sgd, train_gs_reset
from SCM_data import generate_scm_data
import numpy as np
from numpy.linalg import LinAlgError, inv
from scipy.linalg import sqrtm
import MEC

c:\Users\super\DAG\nodag


In [15]:
# CHAIN
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(4,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9905, diff_ratio=0.0005 -> ✅
penalty = 1.5008
[01] likelihood_est=3.0209, diff_ratio=0.0097 -> ❌
penalty = 1.3213
[02] likelihood_est=2.9905, diff_ratio=0.0005 -> ✅
penalty = 1.5018
[03] likelihood_est=2.9961, diff_ratio=0.0014 -> ✅
penalty = 1.5009
[04] likelihood_est=2.9913, diff_ratio=0.0002 -> ✅
penalty = 1.4998
[05] likelihood_est=2.9905, diff_ratio=0.0004 -> ✅
penalty = 1.9952
True result, seed =  6
[06] likelihood_est=2.9969, diff_ratio=0.0017 -> ✅
penalty = 0.9211
True result, seed =  7
[07] likelihood_est=3.0253, diff_ratio=0.0112 -> ❌
penalty = 0.8797
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 2.5006
[09] likelihood_est=3.2477, diff_ratio=0.0855 -> ❌
penalty = 1.5040
[10] likelihood_est=30.7630, diff_ratio=9.2822 -> ❌
penalty = 1.0000
[11] likelihood_est=2.9917, diff_ratio=0.0001 -> ✅
penalty = 1.9882
[12] likelihood_est=2.9914, diff_ratio=0.0002 -> ✅
pe

In [16]:

print("True times:", true_count)
print("True rate:", true_count / t)

True times: 18
True rate: 0.18


In [3]:
# CHAIN, ITER = 10000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(4,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=10000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9942, diff_ratio=0.0008 -> ✅
penalty = 1.4999
[01] likelihood_est=3.0081, diff_ratio=0.0054 -> ❌
penalty = 1.3544
[02] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.5007
[03] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 1.4997
[04] likelihood_est=2.9907, diff_ratio=0.0004 -> ✅
penalty = 1.4996
[05] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 1.9978
True result, seed =  6
[06] likelihood_est=2.9999, diff_ratio=0.0027 -> ✅
penalty = 0.8781
True result, seed =  7
[07] likelihood_est=3.0060, diff_ratio=0.0047 -> ✅
penalty = 0.8424
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 2.5002
[09] likelihood_est=3.2467, diff_ratio=0.0852 -> ❌
penalty = 1.5039
[10] likelihood_est=30.7632, diff_ratio=9.2823 -> ❌
penalty = 1.0000
[11] likelihood_est=2.9916, diff_ratio=0.0001 -> ✅
penalty = 1.9871
[12] likelihood_est=2.9916, diff_ratio=0.0001 -> ✅
pe

In [16]:
# CHAIN, ITER = 50000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(4,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=50000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print(f"Converged runs: {success_count}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.4997
[01] likelihood_est=3.0030, diff_ratio=0.0037 -> ✅
penalty = 1.3607
[02] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.5001
[03] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.4997
[04] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.4996
[05] likelihood_est=2.9902, diff_ratio=0.0005 -> ✅
penalty = 1.9979
True result, seed =  6
[06] likelihood_est=3.0047, diff_ratio=0.0043 -> ✅
penalty = 0.8481
True result, seed =  7
[07] likelihood_est=2.9939, diff_ratio=0.0007 -> ✅
penalty = 0.8490
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 2.5000
[09] likelihood_est=3.2538, diff_ratio=0.0875 -> ❌
penalty = 1.5019
[10] likelihood_est=30.7632, diff_ratio=9.2823 -> ❌
penalty = 1.0000
[11] likelihood_est=2.9914, diff_ratio=0.0002 -> ✅
penalty = 1.9890
[12] likelihood_est=2.9923, diff_ratio=0.0002 -> ✅
pe

In [ ]:
# Collider
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("A_true = \n",A_true)
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9906, diff_ratio=0.0004 -> ✅
penalty = 1.4905
[01] likelihood_est=2.9910, diff_ratio=0.0003 -> ✅
penalty = 2.0001
[02] likelihood_est=3.0007, diff_ratio=0.0030 -> ✅
penalty = 1.9862
[03] likelihood_est=3.2792, diff_ratio=0.0960 -> ❌
penalty = 1.5094
[04] likelihood_est=3.0160, diff_ratio=0.0081 -> ❌
penalty = 1.3819
True result, seed =  5
[05] likelihood_est=3.0911, diff_ratio=0.0332 -> ❌
penalty = 0.6332
True result, seed =  6
[06] likelihood_est=29.6793, diff_ratio=8.9200 -> ❌
penalty = 0.6641
True result, seed =  7
[07] likelihood_est=3.0861, diff_ratio=0.0315 -> ❌
penalty = 0.5992
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2959, diff_ratio=0.1016 -> ❌
penalty = 1.5379
[10] likelihood_est=3.7719, diff_ratio=0.2607 -> ❌
penalty = 1.1335
[11] likelihood_est=2.9981, diff_ratio=0.0021 -> ✅
penalty = 1.9176
[12] likelihood_est=3.2762, di

In [4]:
# Collider, ITER = 10000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=10000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=3.3287, diff_ratio=0.1126 -> ❌
penalty = 1.0186
[01] likelihood_est=2.9902, diff_ratio=0.0005 -> ✅
penalty = 2.0002
[02] likelihood_est=2.9919, diff_ratio=0.0000 -> ✅
penalty = 1.9890
[03] likelihood_est=3.2789, diff_ratio=0.0959 -> ❌
penalty = 1.5097
[04] likelihood_est=3.0088, diff_ratio=0.0057 -> ❌
penalty = 1.4179
True result, seed =  5
[05] likelihood_est=3.1178, diff_ratio=0.0421 -> ❌
penalty = 0.6546
True result, seed =  6
[06] likelihood_est=29.7083, diff_ratio=8.9297 -> ❌
penalty = 0.6264
True result, seed =  7
[07] likelihood_est=3.1286, diff_ratio=0.0457 -> ❌
penalty = 0.5732
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2667, diff_ratio=0.0919 -> ❌
penalty = 1.5411
[10] likelihood_est=3.1251, diff_ratio=0.0445 -> ❌
penalty = 1.5950
[11] likelihood_est=2.9963, diff_ratio=0.0015 -> ✅
penalty = 1.8992
[12] likelihood_est=3.2950, di

In [1]:
# Collider, ITER = 50000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=50000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print(f"Converged runs: {success_count}")

KeyboardInterrupt: 

### No Reset

In [6]:
# CHAIN
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(4,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9906, diff_ratio=0.0004 -> ✅
penalty = 1.5003
[01] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 2.0007
[02] likelihood_est=2.9905, diff_ratio=0.0005 -> ✅
penalty = 1.5018
[03] likelihood_est=2.9961, diff_ratio=0.0014 -> ✅
penalty = 1.5009
[04] likelihood_est=2.9913, diff_ratio=0.0002 -> ✅
penalty = 1.4998
[05] likelihood_est=3.0012, diff_ratio=0.0031 -> ✅
penalty = 1.9978
[06] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 1.9997
True result, seed =  7
[07] likelihood_est=3.0253, diff_ratio=0.0112 -> ❌
penalty = 0.8797
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2477, diff_ratio=0.0855 -> ❌
penalty = 1.5040
[10] likelihood_est=2.9910, diff_ratio=0.0003 -> ✅
penalty = 1.4961
[11] likelihood_est=2.9917, diff_ratio=0.0001 -> ✅
penalty = 1.9882
[12] likelihood_est=2.9910, diff_ratio=0.0003 -> ✅
penalty = 1.5008
[13] like

In [7]:
# CHAIN, ITER = 10000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(4,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=10000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9929, diff_ratio=0.0004 -> ✅
penalty = 1.4999
[01] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 2.0003
[02] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.5007
[03] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 1.4997
[04] likelihood_est=2.9907, diff_ratio=0.0004 -> ✅
penalty = 1.4996
[05] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9993
[06] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9997
True result, seed =  7
[07] likelihood_est=3.0060, diff_ratio=0.0047 -> ✅
penalty = 0.8424
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2467, diff_ratio=0.0852 -> ❌
penalty = 1.5039
[10] likelihood_est=3.0300, diff_ratio=0.0127 -> ❌
penalty = 1.4339
[11] likelihood_est=2.9916, diff_ratio=0.0001 -> ✅
penalty = 1.9871
[12] likelihood_est=2.9923, diff_ratio=0.0001 -> ✅
penalty = 1.5003
[13] like

In [15]:
# CHAIN, ITER = 50000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(4,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=50000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,1],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.4997
[01] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 2.0000
[02] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.5001
[03] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.4997
[04] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.4996
[05] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9996
[06] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9999
True result, seed =  7
[07] likelihood_est=2.9939, diff_ratio=0.0007 -> ✅
penalty = 0.8490
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2538, diff_ratio=0.0875 -> ❌
penalty = 1.5019
[10] likelihood_est=3.0034, diff_ratio=0.0038 -> ✅
penalty = 1.4264
[11] likelihood_est=2.9914, diff_ratio=0.0002 -> ✅
penalty = 1.9890
[12] likelihood_est=2.9904, diff_ratio=0.0005 -> ✅
penalty = 1.4999
[13] like

In [8]:
# Collider
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=2.9906, diff_ratio=0.0004 -> ✅
penalty = 1.4905
[01] likelihood_est=2.9910, diff_ratio=0.0003 -> ✅
penalty = 2.0001
[02] likelihood_est=3.0772, diff_ratio=0.0285 -> ❌
penalty = 1.5023
[03] likelihood_est=3.2792, diff_ratio=0.0960 -> ❌
penalty = 1.5094
[04] likelihood_est=3.2256, diff_ratio=0.0781 -> ❌
penalty = 1.0720
[05] likelihood_est=2.9905, diff_ratio=0.0005 -> ✅
penalty = 1.5009
[06] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9996
True result, seed =  7
[07] likelihood_est=2.9933, diff_ratio=0.0005 -> ✅
penalty = 0.9154
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2959, diff_ratio=0.1016 -> ❌
penalty = 1.5379
[10] likelihood_est=3.8269, diff_ratio=0.2791 -> ❌
penalty = 1.1132
[11] likelihood_est=2.9981, diff_ratio=0.0021 -> ✅
penalty = 1.9176
[12] likelihood_est=3.2762, diff_ratio=0.0950 -> ❌
penalty = 1.5119
True resu

In [9]:
# Collider, ITER = 10000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=10000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=3.3287, diff_ratio=0.1126 -> ❌
penalty = 1.0186
[01] likelihood_est=2.9902, diff_ratio=0.0005 -> ✅
penalty = 2.0002
[02] likelihood_est=3.0775, diff_ratio=0.0286 -> ❌
penalty = 1.5009
[03] likelihood_est=3.2789, diff_ratio=0.0959 -> ❌
penalty = 1.5097
[04] likelihood_est=3.5721, diff_ratio=0.1939 -> ❌
penalty = 0.5673
[05] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 1.5001
[06] likelihood_est=2.9903, diff_ratio=0.0005 -> ✅
penalty = 1.9996
True result, seed =  7
[07] likelihood_est=2.9913, diff_ratio=0.0002 -> ✅
penalty = 0.9259
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.2667, diff_ratio=0.0919 -> ❌
penalty = 1.5411
[10] likelihood_est=3.1587, diff_ratio=0.0558 -> ❌
penalty = 1.5528
[11] likelihood_est=2.9963, diff_ratio=0.0015 -> ✅
penalty = 1.8992
[12] likelihood_est=3.2950, diff_ratio=0.1013 -> ❌
penalty = 1.5067
True resu

In [14]:
# Collider, ITER = 50000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=50000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
[00] likelihood_est=3.5077, diff_ratio=0.1724 -> ❌
penalty = 0.5766
[01] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9999
[02] likelihood_est=3.0779, diff_ratio=0.0288 -> ❌
penalty = 1.5001
[03] likelihood_est=3.2923, diff_ratio=0.1004 -> ❌
penalty = 1.5063
[04] likelihood_est=3.6917, diff_ratio=0.2339 -> ❌
penalty = 0.5124
[05] likelihood_est=2.9902, diff_ratio=0.0005 -> ✅
penalty = 1.5000
[06] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 1.9998
True result, seed =  7
[07] likelihood_est=3.0029, diff_ratio=0.0037 -> ✅
penalty = 0.9194
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=2.9914, diff_ratio=0.0002 -> ✅
penalty = 1.4980
[10] likelihood_est=3.0636, diff_ratio=0.0240 -> ❌
penalty = 1.6153
[11] likelihood_est=3.0006, diff_ratio=0.0029 -> ✅
penalty = 1.9209
[12] likelihood_est=3.2897, diff_ratio=0.0995 -> ❌
penalty = 1.5080
True resu

### X->Y, Z

In [13]:
# X->Y, Z noreset, 5000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(2,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gumbel_sgd(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,0,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print("True rate:", true_count / t)
print(f"Tolerance (relative error): {tol:.2f}")
print(f"Converged runs: {success_count}")
print(f"Convergence ratio: {convergence_ratio:.3f}")
print(f"Average diff ratio: {np.mean(likelihood_diffs):.4f}")
print(f"Median diff ratio: {np.median(likelihood_diffs):.4f}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 0 0]]
likelihood_true =  2.9918682446140235
[00] likelihood_est=2.9961, diff_ratio=0.0014 -> ✅
penalty = 1.9136
[01] likelihood_est=2.9928, diff_ratio=0.0003 -> ✅
penalty = 1.9997
[02] likelihood_est=2.9905, diff_ratio=0.0005 -> ✅
penalty = 1.5018
[03] likelihood_est=2.9909, diff_ratio=0.0003 -> ✅
penalty = 1.4994
[04] likelihood_est=2.9988, diff_ratio=0.0023 -> ✅
penalty = 1.9986
[05] likelihood_est=2.9905, diff_ratio=0.0005 -> ✅
penalty = 1.5009
True result, seed =  6
[06] likelihood_est=2.9916, diff_ratio=0.0001 -> ✅
penalty = 1.4612
[07] likelihood_est=2.9914, diff_ratio=0.0002 -> ✅
penalty = 1.4997
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 3.0000
[09] likelihood_est=3.0105, diff_ratio=0.0062 -> ❌
penalty = 1.4235
[10] likelihood_est=2.9957, diff_ratio=0.0013 -> ✅
penalty = 1.3842
[11] likelihood_est=2.9916, diff_ratio=0.0001 -> ✅
penalty = 1.4994
[12] likelihood_est=2.9919, diff_ratio=0.0000 -> ✅
penalty = 1.5012
True resu

In [12]:
# X->Y, Z , reset, 5000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B


# Load dataset
X, Y, Z, G_true, CPDAG = generate_scm_data(2,10000, seed = 42)
A_true = (np.eye(3) - G_true)
data = np.array([X, Y, Z]).T
R_hat = np.cov(data.T)
print(R_hat.shape)
d = R_hat.shape[0]
Omega_true = np.eye(d)
Theta_hat = inv(R_hat)
# A_init = np.linalg.cholesky(Theta_hat)
# A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
np.random.seed(54)
B_init = np.random.randn(*R_hat.shape)
B_true = G_true
beta = 5
lam = 0.5



A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
print("G_true = \n",G_true)
likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
print("likelihood_true = ", likelihood_true)
true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
f_true = likelihood_true + true_penalty

t = 100
tol = 0.005                 # 收敛阈值（1%以内认为收敛）

likelihood_diffs = []
success_count = 0
true_count = 0
for seed in range(t):
    np.random.seed(seed)
    B_init = np.random.randn(*R_hat.shape)

    B_final, G_final, info = train_gs_reset(
        Rhat_np=R_hat,
        lam=lam,
        delta=1e-6,
        tau_start=1,
        tau_end=1,
        B_init=B_init,
        max_steps=5000,
        history_every=100,
        seed=seed,
        printreset=False
    )

    likelihood_est = info["final_likelihood"]
    penalty_est = info["final_penalty"]

    diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
    likelihood_diffs.append(diff_ratio)
    if (
        np.array_equal(G_final, np.array([[0,0,0],[1,0,0],[0,0,0]]))
        or np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,0,0]]))
        ):  
        true_count += 1
        print("True result, seed = ",seed)
    is_converged = diff_ratio < tol
    success_count += int(is_converged)
    status = "✅" if is_converged else "❌"
    print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
    f"diff_ratio={diff_ratio:.4f} -> {status}")
    
    print(f"penalty = {penalty_est:.4f}")



eps = 1e-6
abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

# 计算 U_init = logit(|B_init|)
U_init = np.log(abs_B / (1 - abs_B))

# 计算 P_init = sign(B_init) * |B_init|
P_init = np.sign(B_init) * abs_B




likelihood_diffs = np.array(likelihood_diffs)
convergence_ratio = success_count / t
print("\n=== Summary ===")
print(f"Total runs: {t}")
print("True times:", true_count)
print(f"Converged runs: {success_count}")

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 0 0]]
likelihood_true =  2.9918682446140235
[00] likelihood_est=2.9966, diff_ratio=0.0016 -> ✅
penalty = 1.4146
[01] likelihood_est=2.9934, diff_ratio=0.0005 -> ✅
penalty = 1.5003
[02] likelihood_est=2.9907, diff_ratio=0.0004 -> ✅
penalty = 1.5018
[03] likelihood_est=2.9909, diff_ratio=0.0003 -> ✅
penalty = 1.4994
[04] likelihood_est=2.9988, diff_ratio=0.0023 -> ✅
penalty = 1.9986
[05] likelihood_est=3.0038, diff_ratio=0.0040 -> ✅
penalty = 1.4125
True result, seed =  6
[06] likelihood_est=2.9922, diff_ratio=0.0001 -> ✅
penalty = 0.4658
[07] likelihood_est=2.9914, diff_ratio=0.0001 -> ✅
penalty = 1.0003
[08] likelihood_est=2.9902, diff_ratio=0.0006 -> ✅
penalty = 2.0013
[09] likelihood_est=3.0105, diff_ratio=0.0062 -> ❌
penalty = 1.4235
[10] likelihood_est=2.9957, diff_ratio=0.0013 -> ✅
penalty = 1.3841
[11] likelihood_est=2.9916, diff_ratio=0.0001 -> ✅
penalty = 1.4994
[12] likelihood_est=2.9921, diff_ratio=0.0001 -> ✅
penalty = 1.0018
True resu

### Test different iters

In [3]:
# Collider, ITER = 100, 500, 1000, 2000, 5000, 10000, 30000
import logging
import numpy as np

from torch.utils.data import Dataset

from synthetic_dataset import SyntheticDataset
from synthetic_dataset import dataset_based_on_B

iters = [100, 500, 1000, 2000, 3000, 5000, 7500, 10000, 30000]
for iter in iters:
    print(f"ITER = {iter}\n")
    # Load dataset
    X, Y, Z, G_true, CPDAG = generate_scm_data(3,10000, seed = 42)
    A_true = (np.eye(3) - G_true)
    data = np.array([X, Y, Z]).T
    R_hat = np.cov(data.T)
    print(R_hat.shape)
    d = R_hat.shape[0]
    Omega_true = np.eye(d)
    Theta_hat = inv(R_hat)
    # A_init = np.linalg.cholesky(Theta_hat)
    # A_init = A_init + 0.01 * np.random.randn(*A_init.shape)
    np.random.seed(54)
    B_init = np.random.randn(*R_hat.shape)
    B_true = G_true
    beta = 5
    lam = 0.5



    A_true = (np.eye(d) - B_true) @ inv(sqrtm(Omega_true))
    print("G_true = \n",G_true)
    likelihood_true = - 2 * np.log(np.linalg.det(A_true)) + np.trace(A_true.T @ R_hat @ A_true)
    print("likelihood_true = ", likelihood_true)
    true_penalty = lam * np.count_nonzero(A_true[~np.eye(A_true.shape[0], dtype=bool)])
    f_true = likelihood_true + true_penalty

    t = 100
    tol = 0.005                 # 收敛阈值（1%以内认为收敛）

    likelihood_diffs = []
    success_count = 0
    true_count = 0
    for seed in range(t):
        np.random.seed(seed)
        B_init = np.random.randn(*R_hat.shape)

        B_final, G_final, info = train_gumbel_sgd(
            Rhat_np=R_hat,
            lam=lam,
            delta=1e-6,
            tau_start=1,
            tau_end=1,
            B_init=B_init,
            max_steps=iter,
            history_every=100,
            seed=seed
        )

        likelihood_est = info["final_likelihood"]
        penalty_est = info["final_penalty"]

        diff_ratio = abs(likelihood_est - likelihood_true) / abs(likelihood_true)
        likelihood_diffs.append(diff_ratio)
        if (
            np.array_equal(G_final, np.array([[0,1,0],[0,0,0],[0,1,0]]))
            ):  
            true_count += 1
            print("True result, seed = ",seed)
        is_converged = diff_ratio < tol
        success_count += int(is_converged)
        status = "✅" if is_converged else "❌"
        # print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
        #  f"diff_ratio={diff_ratio:.4f} -> {status}")
        #print(f"[{seed:02d}] likelihood_est={likelihood_est:.4f}, "
        #f"-2logdet = {-2*info['final_logdet']:.4f}, "
        #f"trace = {np.trace(B_final.T @ R_hat @ B_final):.4f}, "
        #f"diff_ratio={diff_ratio:.4f} -> {status}")
        
        # print(f"penalty = {penalty_est:.4f}")



    eps = 1e-6
    abs_B = np.clip(np.abs(B_init), eps, 1 - eps)

    # 计算 U_init = logit(|B_init|)
    #U_init = np.log(abs_B / (1 - abs_B))

    # 计算 P_init = sign(B_init) * |B_init|
    #P_init = np.sign(B_init) * abs_B

    likelihood_diffs = np.array(likelihood_diffs)
    convergence_ratio = success_count / t
    print("\n=== Summary ===")
    print(f"Total runs: {t}")
    print("True times:", true_count)
    print(f"Converged runs: {success_count}\n\n")

ITER = 100

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243

=== Summary ===
Total runs: 100
True times: 0
Converged runs: 14


ITER = 500

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
True result, seed =  7
True result, seed =  43
True result, seed =  95

=== Summary ===
Total runs: 100
True times: 3
Converged runs: 21


ITER = 1000

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
True result, seed =  7
True result, seed =  13
True result, seed =  25
True result, seed =  43
True result, seed =  73
True result, seed =  93
True result, seed =  95

=== Summary ===
Total runs: 100
True times: 7
Converged runs: 29


ITER = 2000

(3, 3)
G_true = 
 [[0 1 0]
 [0 0 0]
 [0 1 0]]
likelihood_true =  2.9918682446140243
True result, seed =  7
True result, seed =  13
True result, seed =  25
True result, seed =  27
True result, seed =  43
True result, seed =  73
True result, seed =  93
True 